In [1]:
# Utilities
import time
import pandas as pd
import tqdm as tqdm

# Pytorch Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader

import pytorch_lightning as pl

# Huggingface
from transformers import AutoTokenizer, AutoModel, AdamW

# Repository 
from utils import *
from dataset.triples import TriplesDataset
from model.cross_encoder import CrossEncoder
# from trainer.train import Trainer
%load_ext autoreload
%autoreload 2

2022-04-24 18:55:59.645678: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-24 18:55:59.645789: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
CONFIG = {
    'model_name': 'bert-base-uncased',
    'query_maxlen': 64,
    'passage_maxlen': 128,
    'batch_size': 2,
    'epochs': 5,
    'learning_rate': 1e-3,
}

In [3]:
triples = read_triples('data/triples.train.small.tsv', 10)
triples.head()

Reading Chunks: : 55000it [00:00, 109834.50it/s]                                                                                                                                                                   


,query,positive_passage,negative_passage
0,is a little caffeine ok during pregnancy,We donât know a lot about the effects of caf...,It is generally safe for pregnant women to eat...
1,what fruit is native to australia,Passiflora herbertiana. A rare passion fruit n...,"The kola nut is the fruit of the kola tree, a ..."
2,how large is the canadian military,The Canadian Armed Forces. 1 The first large-...,The Canadian Physician Health Institute (CPHI)...
3,types of fruit trees,Cherry. Cherry trees are found throughout the ...,"The kola nut is the fruit of the kola tree, a ..."
4,how many calories a day are lost breastfeeding,"Not only is breastfeeding better for the baby,...","However, you still need some niacin each day; ..."


In [4]:
tokenizer = AutoTokenizer.from_pretrained(CONFIG['model_name'])
triples_dataset = TriplesDataset(triples, tokenizer, CONFIG['query_maxlen'], CONFIG['passage_maxlen'])
triples_dataloader = DataLoader(triples_dataset, batch_size=CONFIG['batch_size'])
batch = next(iter(triples_dataloader))
for key, val in batch.items():
    print(f'{key}: {val.shape}')

cls_id: torch.Size([2])
sep_id: torch.Size([2])
query_ids: torch.Size([2, 64])
query_mask: torch.Size([2, 64])
pos_ids: torch.Size([2, 128])
pos_mask: torch.Size([2, 128])
neg_ids: torch.Size([2, 128])
neg_mask: torch.Size([2, 128])
target: torch.Size([2, 1])


In [5]:
trainer = pl.Trainer(fast_dev_run=True)
model = CrossEncoder(CONFIG)
# optimizer = torch.optim.Adam(params = model.parameters(), lr = 2e-5)
# criterion = nn.MarginRankingLoss()

/home/dminn/Projects/noise-proof-passage-retrieval/passage_retrieval_env/lib/python3.8/site-packages/pytorch_lightning/loops/utilities.py:91: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Running in fast_dev_run mode: will run a full train, val, test and prediction loop using 1 batch(es).
`Trainer(limit_train_batches=1)` was configured so 1 batch per epoch will be used.
`Trainer(limit_val_batches=1)` was configured so 1 batch will be used.
`Trainer(limit_test_batches=1)` was configured so 1 batch will be used.
`Trainer(limit_predict_batches=1)` was configured so 1 batch will be used.
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..


In [6]:
trainer.fit(model=model, train_dataloaders=triples_dataloader)

/home/dminn/Projects/noise-proof-passage-retrieval/passage_retrieval_env/lib/python3.8/site-packages/pytorch_lightning/trainer/configuration_validator.py:133: UserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn("You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.")

  | Name    | Type      | Params
--------------------------------------
0 | encoder | BertModel | 109 M 
1 | fc      | Linear    | 769   
--------------------------------------
109 M     Trainable params
0         Non-trainable params
109 M     Total params
437.932   Total estimated model params size (MB)


Epoch 0:   0%|                                                                                                                                                                               | 0/1 [00:00<?, ?it/s]

/home/dminn/Projects/noise-proof-passage-retrieval/passage_retrieval_env/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/dminn/Projects/noise-proof-passage-retrieval/passage_retrieval_env/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1927: PossibleUserWarning: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 0: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:11<00:00, 11.12s/it, loss=0.0836, v_num=]
